In [82]:
import pyquery
import requests
import urllib
Q = pyquery.PyQuery
import collections

%matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter, FuncFormatter

In [100]:
def get_cars(car_list, results):
    # I suck at using JQuery, so I sloppily make my way down the 
    # HTML chain. The hertz HTML isn't super friendly, but each item seems
    # to have a unique CSS class to seek to. 
    # In a couple places, I hardcode child indices because it was easier
    # than making a new Q object to go down one level. 
    for listing in car_list.getchildren():
        ql = Q(listing)
        desc = ql(".gv-description")[0]
        odo = desc[0][1].text
        mi, unit = odo.split()
        mi = int(mi.replace(",",""))
        year = ql(".inventory-title.single-line-text")[0][0].text.split()[0]
        year = int(year)

        price = ql("span.askingPrice.finalPrice")[0]
        value = price[1].text.replace("\"", "").strip().replace("$", "").replace(",", "")
        value = int(value)
        results[year].append((mi, value))
            
def get_listing(url, params):
    rsp = requests.get(url, params=params)
    content = rsp.content
    return Q(content)

def get_car_list(q):
    return q(".gv-inventory-list")[0]

def get_num_results(q):
    return int(q(".total-count")[0][0].text)

def c2l(cc):
    return list(zip(*cc))

def ann(ax, xx, yy):
    for x, y in zip(xx,yy):
        ax.annotate(y, (x,y))
        
def mmy(params):
    return params['make'],params['model'],params['year']

In [103]:
qp2 = {
     'geoRadius': '500',
     'geoZip': '93012',
     'compositeType': 'certified, Rent2Buy',
     'model': 'Corolla',
     'make': 'Toyota',
     'normalBodyStyle': 'Sedan',
     'normalExteriorColor': 'Black,Gray,Other',
     'odometer': '10-200000',
     'sortBy': 'odometer asc',
     'trim': 'LE',
     'year': '2015-2018',
     'start': '0'
}

In [98]:
qp2['start'] = '0'
pq = get_listing(url, qp2)
nresults = get_num_results(pq)
cars_listing = get_car_list(pq)
got = 0
results = collections.defaultdict(list)
while True:
    got += len(cars_listing)
    print("parsing cars 1-%d of %d"%(got, nresults))
    get_cars(cars_listing, results)
    if got >= nresults:
        break
    qp2['start'] = str(got)
    pq = get_listing(url, qp2)
    cars_listing = get_car_list(pq)

parsing cars 1-35 of 206
parsing cars 1-70 of 206
parsing cars 1-105 of 206
parsing cars 1-140 of 206
parsing cars 1-175 of 206
parsing cars 1-206 of 206


In [105]:
# NameError card so it runs correctly on first execution
try:
    fig.clear()
    plt.close(fig)
except NameError: 
    pass

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

for year, cars in sorted(results.items(), key=lambda item: item[0], reverse=True): # sort by year
    x, y = c2l(cars)
    ax.scatter(x,y, label=str(year))

# Add "," separator and $ sign
xf = FuncFormatter(lambda x, p: format(int(x), ','))
yf = FuncFormatter(lambda y, p: "$%s"%format(int(y), ","))
ax.xaxis.set_major_formatter(xf)
ax.yaxis.set_major_formatter(yf)

ax.xaxis.set_label_text("Miles Driven")
ax.yaxis.set_label_text("Price (No haggle)")

ax.set_title("%s %s %s"%(mmy(qp2)))

# Yaxis label doesn't quite fit, so fudge it over a bit
b = ax.get_position()
shift = 0.02
ax.set_position([b.x0+shift, b.y0, b.width*(1-shift), b.height])

# assign just so it doesn't get stuck in global Jupyter memory
_=ax.legend()

In [106]:
ax.grid()